In [170]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import datetime as dt
from numpy import mean
from numpy import std

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import KNNImputer

# encoders
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

# classifiers
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, StackingClassifier, VotingClassifier, GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import lightgbm as lgb
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn import svm

# hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from sklearn.decomposition import PCA
from matplotlib.pyplot import figure

# 1. EDA

In [269]:
features = pd.read_csv("train_val.csv")
labels = pd.read_csv("train_label.csv")
test = pd.read_csv("test.csv")

In [272]:
X = features.copy()
y = labels.copy()['status_group']
X_test = test.copy()

In [273]:
df = pd.concat([X,X_test],axis=0)

## A. Delete columns

In [274]:
X_null = X.isnull().sum()
test_null = X_test.isnull().sum()

In [275]:
# 변동성이 없는 컬럼 삭제
columns_to_drop1 = [col for col in X.columns if X[col].nunique() == 1]

In [276]:
# 70% 이상 값이 0인 컬럼 삭제
num_of_row = X.shape[0]
X_zero = X.astype(bool).sum(axis=0)
col_X = X.columns

i = 0
for col in col_X:
  if (((num_of_row - X_zero.values[i])/num_of_row)*100 > 0):
    print('feature:', col, ((num_of_row - X_zero.values[i])/num_of_row)*100 )
  i += 1

columns_to_drop2 = ['amount_tsh','num_private']

feature: id 0.0016835016835016834
feature: amount_tsh 70.09932659932659
feature: gps_height 34.40740740740741
feature: longitude 3.05050505050505
feature: num_private 98.72558922558923
feature: district_code 0.038720538720538725
feature: population 35.994949494949495
feature: public_meeting 8.51010101010101
feature: permit 29.44781144781145
feature: construction_year 34.86363636363636


In [180]:
# 고유한 이름을 가지는 컬럼 삭제
columns_to_drop3 = ['scheme_name','wpt_name']

In [181]:
remove_columns = columns_to_drop1+columns_to_drop2+columns_to_drop3
df = df.drop(remove_columns,axis=1)

**변수 삭제 완료!**

## B. 결측치 처리

In [182]:
missing_val_count_by_column = (df.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

funder               4507
installer            4532
subvillage            470
public_meeting       4155
scheme_management    4847
permit               3793
dtype: int64


- `Population` - 36% 정도의 값이 0이다. 결측치 처리 해야함 -> 평균으로 처리
- `funder` - 최빈값
- `installer` - 최빈값
- `subvilage` - 최빈값
- `public_meeting` - 최빈값
- `scheme_management` - 최빈값
- `permit` - 최빈값
- `construction year` - 날짜가 0이므로 Nan으로 바꾼다. 

In [183]:
mis_columns = ['construction_year','population']
[df[col].replace(0, np.nan) for col in mis_columns]

[0        1999.0
 1        2010.0
 2        2009.0
 3        1986.0
 4           NaN
           ...  
 14845    1988.0
 14846    1994.0
 14847    2010.0
 14848    2009.0
 14849    2008.0
 Name: construction_year, Length: 74250, dtype: float64,
 0         109.0
 1         280.0
 2         250.0
 3          58.0
 4           NaN
           ...  
 14845      20.0
 14846    2960.0
 14847     200.0
 14848     150.0
 14849      40.0
 Name: population, Length: 74250, dtype: float64]

In [184]:
# 최빈값으로 채우는 함수
def fill_freq(all_column,X):
  for col in all_column:
    X[col] = X[col].fillna(X[col].mode()[0])
  return X

In [185]:
df = fill_freq(["funder","installer","subvillage","public_meeting","scheme_management","permit","construction_year"],df)

/tmp/ipykernel_94134/748309975.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[col] = X[col].fillna(X[col].mode()[0])


In [186]:
# 평균으로 채우는 함수
def fill_mean(all_column,X):
  for X_column in all_column:
    X[X_column] = X[X_column].fillna(round(X[X_column].mean()))
  return X

In [187]:
df = fill_mean(["population"],df)

**결측치 처리 완료!**

## C. Data Preprocessing

In [188]:
df.date_recorded = pd.to_datetime(df.date_recorded)

df['year'] = df['date_recorded'].dt.year
df['month'] = df['date_recorded'].dt.month
df['day_of_months'] = df['date_recorded'].dt.day
df.drop(columns=['date_recorded'], axis=1, inplace=True)

In [189]:
#Payment
df["Payment_all"] = df["payment"] + df["payment_type"]

#quality
df["Quality_all"] = df["water_quality"] + df["quality_group"]

#Quantitiy
df["Quantity_all"] = df["quantity"] + df["quantity_group"]

#waterpoint_type
df["Waterpoint_all"] = df["waterpoint_type"] + df["waterpoint_type_group"]

In [190]:
df.drop(columns=['payment','payment_type','water_quality','quality_group','quantity','quantity_group','waterpoint_type','waterpoint_type_group'], axis=1, inplace=True)

## D. Encoding

In [191]:
def one_hot_encode(df,object_cols):
  OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
  OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(df[object_cols]))
  OH_cols_train.index = df.index
  num_X_train = df.drop(object_cols, axis=1)
  df = pd.concat([num_X_train, OH_cols_train], axis=1)
  return df

In [192]:
# 고유값이 적은 경우는 one_hot_encode 사용
columns=['source','source_type','source_class','public_meeting','permit','extraction_type','extraction_type_group','extraction_type_class','management','management_group',"basin", "scheme_management","Payment_all","Quality_all","Quantity_all","Waterpoint_all"]
df = one_hot_encode(df, columns)

In [193]:
# 고유값이 많은 경우는 BinaryEncoder 사용
enc = ce.BinaryEncoder(cols=['funder','installer', 'subvillage', 'region','lga','ward']).fit(df)
df = enc.transform(df)

# 2. Separate training and test set

In [203]:
df.columns = df.columns.astype(str)
X = df[:len(features_df)]
X_test = df[len(features_df):]

In [228]:
dct = {'functional' : 0 , 'functional needs repair' : 1 , 'non functional' : 2}
y = y.map(dct)

# 3. Model Training

In [199]:
def score_dataset(X, y, model):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)
    score = cross_val_score(
        model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    print('Accuracy: %.3f (%.3f)' % (mean(score), std(score)))

In [209]:
RF = RandomForestClassifier(n_estimators=400, random_state=0,max_depth=20)

In [230]:
# score model
scoreRFTest = score_dataset(X,y,RF)

Accuracy: 0.816 (0.004)


In [233]:
def grid_search(model, params, X_tune, y_tune):
  clf =  GridSearchCV(model, params, cv=5, scoring='accuracy')
  clf.fit(X_tune, y_tune)
  return clf.best_score_, clf.best_params_

`-` Train on entire dataset

In [242]:
model = modelRF
X_train = X
y_train = y

model.fit(X_train, y_train)

RandomForestClassifier(max_depth=20, n_estimators=400, random_state=0)

# 4. Predict

In [245]:
predict=model.predict(X_test)

In [246]:
predict

array([2, 0, 0, ..., 0, 0, 2], shape=(14850,))

# 5. Submission

In [259]:
sub = pd.read_csv("SubmissionFormat.csv")

In [261]:
dct2 = {v:k for k,v in dct.items()}
sub['status_group'] = pd.Series(predict).map(dct2)

In [267]:
sub.to_csv('my_sub.csv', index=False)